In [ ]:
# Needed depenecies

!pip install h5py
!pip install tensorflow-gpu
!pip install wandb

In [ ]:
# Imports

import h5py
import wandb
import numpy as np
import cv2
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from wandb.keras import WandbCallback
tf.test.gpu_device_name()

## Loading data

In [ ]:
# Connecting to google drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Fetch data from drive

training_data = h5py.File('/content/drive/My Drive/Colab Notebooks/ruap/Data/data.h5', 'r')

In [ ]:
# Checking if data loaded correctly
print(list(training_data.keys()))

In [ ]:
# Loading data into variables

x_data = training_data['x_data'][()]
y_data = training_data['y_data'][()]

In [ ]:
# Shuffling data

np.random.seed(75)
np.random.shuffle(x_data)
np.random.seed(75)
np.random.shuffle(y_data)

In [ ]:
# Spliting data into train, val, test (60%, 25%, 15%)

x_train = x_data[0:4800]
y_train = y_data[0:4800]

x_val = x_data[4800:6400]
y_val = y_data[4800:6400]

x_test = x_data[6400:]
y_test = y_data[6400:]

In [ ]:
#Data augmentation (finnal version Augmentation 5)

train_datagen = ImageDataGenerator(
    width_shift_range=[-20, 20],
    height_shift_range=[-20, 20],
    horizontal_flip=True
    )

train_data = train_datagen.flow(x_train, y_train, batch_size=64)

## Models

In [ ]:
# Function for creating MobileNet instance

def create_MobileNet():

    # Fetching MobileNetV2 from Tensorflow lib
    mobileNet = tf.keras.applications.MobileNetV2(input_shape=(224, 224, 3))

    # Disabling training for all layers
    for (index, layer) in enumerate(mobileNet.layers):
      mobileNet.layers[index].trainable = False

    # Adding my layers on top of MobileNetV2 (Best result model)
    model = tf.keras.Sequential([
        mobileNet,
        tf.keras.layers.Dense(1024, activation='relu'),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(37, activation='softmax')
        ])

    # Compile parameter
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
        )

    return model

In [ ]:
# Function for creating custom model instance

def create_customModel():

  # Model consists of layers put together sequentially
  model = tf.keras.Sequential()

  # Convolutional layers (CNN) and MaxPooling layers
  model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)))
  model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'))
  model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(tf.keras.layers.Dropout(0.5))

  # Layer for flattening CNN's
  model.add(tf.keras.layers.Flatten())

  # Basic layers
  model.add(tf.keras.layers.Dense(256, activation='relu'))
  model.add(tf.keras.layers.Dropout(0.5))
  model.add(tf.keras.layers.Dense(37, activation='softmax'))

  # Compile parameters
  model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy,
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
  
  return model

## Running model

In [ ]:
# Creating model and printing model summary

model = create_customModel()
model.summary()

In [ ]:
#Initialize run on wandb

run = wandb.init(project="MackePsi", entity="samantadeskar", name="cnn-32-64-128-256-(3,3)-256-dropout-augm5-fin")

In [ ]:
# Training

epochs = 50
batch_size = 64
model.fit(train_data, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val), callbacks=[WandbCallback()])

In [ ]:
#Logging accuracy on test data

wandb.log({"Test image accuracy":model.evaluate(x_test, y_test, verbose=1)[1]})

In [ ]:
#Convert and save model for tensorflow.js

!pip install tensorflowjs
import tensorflowjs as tfjs
tfjs.converters.save_keras_model(model, "/content/drive/My Drive/Colab Notebooks/ruap/Model/cnn-32-64-128-256-(3,3)-256-dropout-augm5-fin")